# Import Packages

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u3933826/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from ALPackage.transformer import EMBALLayer, TransformerEncoderALLayer, TransformerEncoder
from utils import get_word_vector, get_nlp_data, set_device
from tqdm import tqdm
import time
import torch.nn as nn
import torch
import pandas as pd
import threading
import numpy as np
from collections import defaultdict

In [3]:
import matplotlib.pyplot as plt

# Prepare data

In [4]:
class Args:
    def __init__(self, dataset, max_len, min_len = 1):
        self.dataset = dataset
        self.class_num = None
        self.batch_size = 256        
        self.max_len = max_len
        self.min_len = min_len
        self.vocab_size = 30000
        self.pretrained_embedding = None
        self.pretrained_model = None
        self.embedding_dim = 300
        self.x_hid = 256
        self.y_hid = 128
        self.n_heads = 6       
        self.dropout = 0.3
        self.lr = 0.00025
        self.epochs = 5
        self.act = nn.Tanh()

    def get_filename(self):
        name = f"{self.dataset},{self.batch_size},{self.max_len},{self.min_len},{self.x_hid},{self.y_hid}"
        return name
    
    def set_pretrained(self, vocab, emb):
        self.pretrained_embedding = get_word_vector(vocab, emb)
        self.pretrained_model = emb

In [5]:
args = Args('ag_news', 128)
train_loader, test_loader, class_num, vocab = get_nlp_data(args)
args.class_num = class_num
args.set_pretrained(vocab, emb='glove')

|========================= Load data:ag_news ========================================================|


Found cached dataset ag_news (/home/u3933826/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Found cached dataset ag_news (/home/u3933826/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


|========================= Preprocessing ============================================================|
Original Data: 120000
Valid Data: 120000
total count words 102019
vocab size 30000


loading glove vocabs...: 100%|██████████| 400000/400000 [00:05<00:00, 72221.09it/s]


found 26754 words in glove


# Scale Test helper

In [6]:
class Transformer(nn.Module):
    def __init__(self, args, tr_num):
        super().__init__()
        self.seq = nn.ModuleList()
        if args.pretrained_embedding == None:
            self.seq.append(nn.Embedding(args.vocab_size, args.embedding_dim))
        else:
            self.seq.append(nn.Embedding.from_pretrained(args.pretrained_embedding, freeze=False))
        for _ in range(tr_num):        
            self.seq.append(TransformerEncoder(args.embedding_dim, args.x_hid, args.n_heads, 1, args.dropout))
        self.seq.append(nn.Sequential(
            nn.Linear(args.embedding_dim, args.class_num),
            nn.LogSoftmax(dim=1)
        ))
        self.device = ["cpu" for _ in range(len(self.seq))]

    def forward(self, x):
        mask = self.get_mask(x)

        # 1. Feed embedding layer
        x = self.seq[0](x.to(self.device[0]))

        # 2. Feed Transformer
        for i in range(1, len(self.seq)-1, 1):
            x = self.seq[i](x.to(self.device[i]), mask.to(self.device[i]))
        
        # 3. Feed fc layer
        x = x.to(self.device[-1])
        mask = mask.to(self.device[-1])
        feat = torch.sum(x*mask.unsqueeze(-1), dim=1) / torch.sum(mask, -1, keepdim=True)
        return self.seq[-1](feat)

    def get_mask(self, x):
        pad_mask = ~(x == 0)
        return pad_mask

In [7]:
class TransformerAL(nn.Module):
    def __init__(self, args, tr_num):
        super().__init__()
        self.seq = nn.ModuleList()
        self.seq.append(EMBALLayer(args.vocab_size, args.embedding_dim, args.class_num, args.y_hid, args.act, args.lr, args.pretrained_embedding))
        for _ in range(tr_num):
            self.seq.append(TransformerEncoderALLayer(args.embedding_dim, args.x_hid, args.y_hid, args.y_hid, args.act, args.lr, args.n_heads, 1, args.dropout))
        self.device = ["cpu" for i in range(len(self.seq))]

    def forward(self, x, y):
        mask = self.get_mask(x)

        # 1. Feed embedding layer
        x, y = self.seq[0](x.to(self.device[0]), y.to(self.device[0]))
        
        # 2. Feed Transformer
        for i in range(1, len(self.seq), 1):
            inputs = (x.to(self.device[i]), mask.to(self.device[i]))
            y = y.to(self.device[i])
            x, y = self.seq[i](inputs, y)

    def backward(self):
        for i in range(len(self.seq)):
            self.seq[i].backward()

    def update(self):
        for i in range(len(self.seq)):
            self.seq[i].update()

    def thread_backward_and_update(self):
        threads = []
        for i in range(len(self.seq)):
            threads.append(threading.Thread(target=self.seq[i].backward_and_update))
            threads[i].start()
        for i in range(len(self.seq)):
            threads[i].join()

    def thread_forward_backward_and_update(self, x, y):
        threads = []
        mask = self.get_mask(x)

        # 1. Feed embedding layer
        x, y = self.seq[0](x.to(self.device[0]), y.to(self.device[0]))
        threads.append(threading.Thread(target=self.seq[0].backward_and_update))
        threads[0].start()

        # 2. Feed Transformer
        for i in range(1, len(self.seq), 1):
            inputs = (x.to(self.device[i]), mask.to(self.device[i]))
            y = y.to(self.device[i])
            x, y = self.seq[i](inputs, y)
            threads.append(threading.Thread(target=self.seq[i].backward_and_update))
            threads[i].start()
        
        for i in range(len(self.seq)):
            threads[i].join()
    
    def loss(self):
        loss = {
            "enc_loss": [],
            "ae_loss": []
        }
        for i in range(len(self.seq)):
            enc_loss, ae_loss = self.seq[i].loss()
            loss['enc_loss'].append(enc_loss.item())
            loss['ae_loss'].append(ae_loss.item())
        return loss

    def inference(self, x):
        mask = self.get_mask(x)
        x = self.seq[0].inference(x.to(self.device[0]), 'f')
        for i in range(1, len(self.seq), 1):
            inputs = (x.to(self.device[i]), mask.to(self.device[i]))
            x = self.seq[i].inference(inputs, 'f')
        inputs = (x, mask.to(self.device[i]))
        x = self.seq[-1].inference(inputs, 'b')
        for i in range(len(self.seq)-1, -1, -1):
            x = x.to(self.device[i])
            x = self.seq[i].inference(x, 'h')
        return x

    def get_mask(self, x):
        pad_mask = ~(x == 0)
        return pad_mask

# Trainer

In [8]:
def Trainer(args, mode, device, tr_num):
    
    train_time = []
    
    if mode == "BP":
        model = Transformer(args, tr_num)
        set_device(model, device)
        optimizer = torch.optim.Adam(model.parameters(), args.lr)
        loss_fn = nn.NLLLoss()
        
        for epoch in range(args.epochs):
            model.train()
            torch.cuda.synchronize()
            start = time.time()
            for inputs, labels in tqdm(train_loader):            
                labels = labels.to(device[-1])
                pred = model(inputs)
                loss_fn(pred, labels).backward()
                optimizer.step()
                optimizer.zero_grad()
            torch.cuda.synchronize()
            end = time.time()
            train_time.append(end-start)
                
    elif mode == "AL":
        model = TransformerAL(args, tr_num)
        set_device(model, device)
        for epoch in range(args.epochs):
            model.train()
            torch.cuda.synchronize()
            start = time.time()
            for inputs, labels in tqdm(train_loader):
                y = torch.nn.functional.one_hot(labels, args.class_num).float()
                model.thread_forward_backward_and_update(inputs, y)
            torch.cuda.synchronize()
            end = time.time()
            train_time.append(end-start)
    
    return np.mean(train_time), np.std(train_time)     

# Scaling Test

In [10]:
def Scale():
    
    '''
    Weak: Add layers, add GPU
    x: number of layers
    y: execution time
    method: gpu_num

    1 gpu with 1x layers
    2 gpu with 2x layers
    ...
    6 gpu with 6x layers 

    ===================================

    Strong: Fix layers, add gpu
    x: number of gpu
    y: execution time
    method: number of layers

    6 layers with 1, 2, 3, 6 gpus
    12 layers with 1, 2, 3, 4, 6 gpus
    '''    
    
    gpus = {
        "1":[[0],[0,0],[0,0,0],[0,0,0,0],[0,0,0,0,0],[0,0,0,0,0,0],
             [0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0]],
        "2":[[0,1],[0,0,1,1],[0,0,0,1,1,1],[0,0,0,0,1,1,1,1],[0,0,0,0,0,1,1,1,1,1],[0,0,0,0,0,0,1,1,1,1,1,1]],
        "3":[[0,1,2],[0,0,1,1,2,2],[0,0,0,1,1,1,2,2,2],[0,0,0,0,1,1,1,1,2,2,2,2]],
        "4":[[0,1,2,3],[0,0,1,1,2,2,3,3],[0,0,0,1,1,1,2,2,2,3,3,3]],
        "5":[[0,1,2,3,4],[0,0,1,1,2,2,3,3,4,4]],
        "6":[[0,1,2,3,4,5],[0,0,1,1,2,2,3,3,4,4,5,5]]
    }

    key = []
    bp = []
    al = []
    
    try:
        mode = "BP"
        for gpu, layers in gpus.items():
            for layer in layers:
                n = len(layer)
                device = ['cuda:0']
                for l in layer:
                    device.append(f"cuda:{l}")
                device.append(f"cuda:{layer[-1]}")
                mean, std = Trainer(args, mode, device, n)
                bp.append(f"{mean:.4f}±{std:.4f}")
                print(f"BP {gpu}gpu_{n}Trlayers:{mean:.4f}±{std:.4f}")
            
        mode = "AL"    
        for gpu, layers in gpus.items():
            for layer in layers:
                n = len(layer)
                device = ['cuda:0']
                for l in layer:
                    device.append(f"cuda:{l}")
                mean, std = Trainer(args, mode, device, n)
                al.append(f"{mean:.4f}±{std:.4f}")
                print(f"AL {gpu}gpu_{n}Trlayers:{mean:.4f}±{std:.4f}")

        for gpu, layers in gpus.items():
            for layer in layers:
                n = len(layer)
                key.append(f"{gpu}gpu_{n}Trlayers")

        result = {
            "key": key,
            "bp": bp,
            "al": al
        }  

        df = pd.DataFrame(result)
        df.to_csv(f"Scale.csv", index=False)
        return bp, al

    except Exception as e:
        print(e)      
        return bp, al

In [11]:
bp, al = Scale()

100%|██████████| 469/469 [00:08<00:00, 56.29it/s]


BP 1gpu_1Trlayers:8.3734±0.0970


100%|██████████| 469/469 [00:14<00:00, 32.59it/s]


BP 1gpu_2Trlayers:14.9438±0.4013


100%|██████████| 469/469 [00:20<00:00, 22.53it/s]


...

AL 5gpu_10Trlayers:56.4686±0.2248


100%|██████████| 469/469 [00:27<00:00, 16.80it/s]


AL 6gpu_6Trlayers:26.8651±0.5387


100%|██████████| 469/469 [01:08<00:00,  6.82it/s]

AL 6gpu_12Trlayers:68.4918±0.2329


# Preview

In [11]:
def CalculateSpeedUp(time):
    speedup = []
    for t in time:
        speedup.append(time[0]/t)
    return speedup

In [12]:
def plotGraph(bp, al, title):
    
    if len(bp)==6:
        gpu = [1, 2, 3, 4, 5, 6]
    else:
        gpu = [1, 2, 3, 4, 6]
    
    fig, axs = plt.subplots(2, 1, figsize=(8, 6))
    fig.suptitle(title)

    # 繪製子圖A
    axs[0].plot(gpu, bp, label="BP")
    axs[0].plot(gpu, al, label="AL")
    axs[0].set_ylabel('Execution Time')
    axs[0].set_xlabel('Number of GPUs')
    axs[0].legend()

    # 繪製子圖B
    axs[1].plot(gpu, CalculateSpeedUp(bp), label="BP")
    axs[1].plot(gpu, CalculateSpeedUp(al), label="AL")
    axs[1].set_ylabel('Speedup')
    axs[1].set_xlabel('Number of GPUs')
    axs[1].legend()

    # 顯示圖表
    plt.savefig(f"{title}.pdf", format="pdf")
    plt.show()   

In [13]:
def readResults(filename):
    
    df = pd.read_csv(filename)
    bp = []
    al = []
    for _, row in df.iterrows():
        bp.append(float(row["bp"].split("±")[0]))
        al.append(float(row["al"].split("±")[0]))
        
    plotGraph(bp, al, filename.split(".csv")[0])        

# WeakScale

In [ ]:
def WeakScalePreview():
    '''
    Graph 1: Time/Speedup(y) with different number of layers/gpus(x)
    Data:
    (1,1),(2,2),(3,3),(4,4),(5,5),(6,6)

    Chart 1: Time/Speedup(BPvsAL) with different number of layers/gpus

    Graph 2: Different number of gpus(label), Time(y) with different number of layers(x)
    Data:
    (1,1~12)
    (2,2~12)
    (3,3~12)
    (6,6~12)
    '''

    data = pd.read_csv("Scale.csv", index_col = 1).to_dict("index")

    # graph 1 & chart 1
    bp = []
    al = []
    for i in range(1,7,1):
        key = f"{i}gpu_{i}Trlayers"
        bp.append(data[key]["bp"].split("±")[0])
        al.append(data[key]["al"].split("±")[0])
    plotGraph(bp, al, "WeakScale.pdf")

    # graph 2
    bp = defaultdict(list)
    al = defaultdict(list)
    for i in range(1,7,1):
        for j in range(i, 13, i):
            key = f"{i}gpu_{j}Trlayers"
            bp[i].append(data[key]["bp"].split("±")[0])
            al[i].append(data[key]["al"].split("±")[0])
    
    plt.plot([i for i in range(1,13,1)], bp[1], label="BP with 1GPU")
    plt.plot([i for i in range(2,13,2)], bp[2], label="BP with 2GPU")
    plt.plot([i for i in range(3,13,3)], bp[3], label="BP with 3GPU")
    plt.plot([i for i in range(6,13,6)], bp[6], label="BP with 6GPU")
    plt.plot([i for i in range(1,13,1)], al[1], label="AL with 1GPU")
    plt.plot([i for i in range(2,13,2)], al[2], label="AL with 2GPU")
    plt.plot([i for i in range(3,13,3)], al[3], label="AL with 3GPU")
    plt.plot([i for i in range(6,13,6)], al[6], label="AL with 6GPU")
    plt.set_ylabel('Execution Time')
    plt.set_xlabel('Number of Encoders')
    plt.legend()
    plt.savefig("WeakScale2.pdf", format="pdf")
    plt.show()

In [ ]:
def StrongScalePreview():
    '''
    Graph 1: Time/Speedup(y) with different number of gpus(x)
    Data:
    (1,12),(2,12),(3,12),(4,12),(6,12)

    Chart 1: Time/Speedup(BPvsAL) with different number of gpus

    Graph 2: Different number of layers(label), Time(y) with different number of gpus(x)
    Data:
    (1,4), (2,4), (4,4)
    (1,6), (2,6), (3,6), (6,6)
    (1,8), (2,8), (4,8)
    (1,12), (2,12), (3,12), (4,12), (6,12)
    '''

    data = pd.read_csv("Scale.csv", index_col = 1).to_dict("index")

    # graph 1 & chart 1
    bp = []
    al = []
    for i in [1,2,3,4,6]:
        key = f"{i}gpu_12Trlayers"
        bp.append(data[key]["bp"].split("±")[0])
        al.append(data[key]["al"].split("±")[0])
    plotGraph(bp, al, "StrongScale.pdf")

    # graph 2
    bp = defaultdict(list)
    al = defaultdict(list)
    for i in [4,6,8,12]:
        for j in [1,2,3,4,6]:
            if i % j == 0:
                key = f"{j}gpu_{i}Trlayers"
                bp[i].append(data[key]["bp"].split("±")[0])
                al[i].append(data[key]["al"].split("±")[0])
    
    plt.plot([1,2,4], bp[4], label="BP with 4Layers")
    plt.plot([1,2,3,6], bp[6], label="BP with 6Layers")
    plt.plot([1,2,4], bp[8], label="BP with 8Layers")
    plt.plot([1,2,3,4,6], bp[12], label="BP with 12Layers")
    plt.plot([1,2,4], al[4], label="AL with 4Layers")
    plt.plot([1,2,3,6], al[6], label="AL with 6Layers")
    plt.plot([1,2,4], al[8], label="AL with 8Layers")
    plt.plot([1,2,3,4,6], al[12], label="AL with 12Layers")
    plt.set_ylabel('Execution Time')
    plt.set_xlabel('Number of GPUs')
    plt.legend()
    plt.savefig("StrongScale2.pdf", format="pdf")
    plt.show()